In [0]:
from torch.utils.data import DataLoader
from torch.autograd import Variable
import numpy as np
import os
import json
import matplotlib.pyplot as plt


json_dir = '/content/drive/Shared drives/Music_Style_Transform/json_sample'
save_dir_b = '/content/drive/My Drive/ADV_Project_Music_style_transform/timbreTron_cycleGAN/cqt_sample/bass'
save_dir_d = '/content/drive/My Drive/ADV_Project_Music_style_transform/timbreTron_cycleGAN/cqt_sample/drum'
save_dir_o = '/content/drive/My Drive/ADV_Project_Music_style_transform/timbreTron_cycleGAN/cqt_sample/other'
save_dir_v = '/content/drive/My Drive/ADV_Project_Music_style_transform/timbreTron_cycleGAN/cqt_sample/vocal'

In [2]:
!pip install librosa==0.7.2

import librosa
import librosa.display

     |████████████████████████████████| 1.6MB 3.4MB/s 
  Created wheel for librosa: filename=librosa-0.7.2-cp36-none-any.whl size=1612885 sha256=c6507491481381f9e786a00834bb1ebdcce30f5fd3dd8af709ad77df95b4ec02
  Stored in directory: /root/.cache/pip/wheels/4c/6e/d7/bb93911540d2d1e44d690a1561871e5b6af82b69e80938abef
Successfully built librosa
  Found existing installation: librosa 0.6.3
    Uninstalling librosa-0.6.3:
      Successfully uninstalled librosa-0.6.3


In [0]:
# json -> cqt


for i,(dirpath,dirnames,filenames) in enumerate(os.walk('/content/drive/Shared drives/Music_Style_Transform/json_sample/Pop')):
    if dirpath is not json_dir:

        genre = dirpath.split('/')[-2]
        print("\n Processing :{}_{}".format(genre,filenames[1:7]))
        
        for index, f in enumerate(filenames):
            file_path = os.path.join(dirpath,f)
            id = f[1:6]
            seg = f[7]
            

                if 'all' not in file_path:
                    f = open(file_path,encoding='UTF-8')
                    data = json.loads(f.read())
                    data = np.array(data['wave'])
                    data = data.reshape(480000)
                    data = np.abs(librosa.core.cqt(data, 32000))
                    data = data[:,:,np.newaxis]
                    if 'bass' in file_path:
                        np.save(os.path.join(save_dir_b,'{}_{}_bass_{}_cqt'.format(genre,id,seg)),data)
                        # print(genre, id, seg)
                        # print(ba.shape)
                    if 'drums' in file_path:
                        # print(da.shape)
                        np.save(os.path.join(save_dir_d,'{}_{}_drum_{}_cqt'.format(genre,id,seg)),data)
                        # print(genre, id, seg)
                    if 'other' in file_path:
                        np.save(os.path.join(save_dir_o,'{}_{}_other_{}_cqt'.format(genre,id,seg)),data)
                        # print(genre, id, seg)
                        # print(oa.shape)
                    if 'vocals' in file_path:
                        np.save(os.path.join(save_dir_v,'{}_{}_vocal_{}_cqt'.format(genre,id,seg)),data)
                        # print(genre, id, seg)
                        # print(va.shape)
                            

# shape : (1, 480000) -> (84, 938)

In [0]:
lslst_data = os.listdir('/content/drive/My Drive/ADV_Project_Music_style_transform/timbreTron_cycleGAN/cqt_sample/other')

In [0]:
lst_data_jazz = [f for f in lslst_data if f.startswith('J')]
lst_data_pop = [f for f in lslst_data if f.startswith('P')]


In [0]:
print(len(lst_data_jazz),len(lst_data_pop))

397 397


In [0]:
jazz = np.load('/content/drive/My Drive/ADV_Project_Music_style_transform/timbreTron_cycleGAN/cqt_sample/other/Jazz_01022_other_1_cqt.npy')

In [0]:
jazz.shape

(84, 938, 1)

In [0]:
def cqtshow(x):    
    librosa.display.specshow(librosa.amplitude_to_db(x.squeeze(), ref=np.max),sr=32000, x_axis='time', y_axis='cqt_note')
    plt.colorbar(format='%+2.0f dB')
    plt.tight_layout()

In [0]:
y_inv = librosa.griffinlim_cqt(data, sr=32000, bins_per_octave=12,hop_length=256,n_iter=50)

In [0]:
import IPython.display as ipd
ipd.Audio(y_inv, rate = 32000) 

In [0]:
f = open('/content/drive/Shared drives/Music_Style_Transform/json_sample/Pop/042048/042048_1_all.json',encoding='UTF-8')
data = json.loads(f.read())
audio = np.array(data)
audio = audio.reshape(480000)
data = np.abs(librosa.core.cqt(audio, 32000,n_bins=144,bins_per_octave=36,hop_length=256))
data.shape

(144, 1876)

In [0]:
f = open('/content/drive/Shared drives/Music_Style_Transform/json_sample/Pop/042048/042048_1_bass.json',encoding='UTF-8')
data = json.loads(f.read())
bass = np.array(data['wave'])
bass = bass.reshape(480000)

In [0]:
ba = bass + audio

In [0]:
f = open('/content/drive/Shared drives/Music_Style_Transform/json_sample/Pop/042048/042048_1_drums.json',encoding='UTF-8')
data = json.loads(f.read())
drum = np.array(data['wave'])
drum = drum.reshape(480000)

In [0]:
da = 0.7 * drum + 0.3 * audio

In [0]:
nov = bass + 0*drum + audio + vocal

In [0]:
f = open('/content/drive/Shared drives/Music_Style_Transform/json_sample/Pop/042048/042048_1_vocals.json',encoding='UTF-8')
data = json.loads(f.read())
vocal = np.array(data['wave'])
vocal = vocal.reshape(480000)

# Wave -> Spectrogram -> Wave 
##Resolution check

In [0]:
sig, sr = librosa.load('/content/drive/My Drive/ADV_Project_Music_style_transform/IU-삐삐.wav')

In [51]:
import IPython.display as ipd
ipd.Audio(sample, rate = sr) 

In [0]:
sample = sig[3200000 : 3280000]

In [0]:
cqt = np.abs(librosa.cqt(sample,sr,hop_length=512,n_bins=80,bins_per_octave=12))

In [67]:
cqt.shape

(80, 157)

In [0]:
cqt_inv = librosa.griffinlim_cqt(cqt,sr,hop_length=512,bins_per_octave=12)

In [0]:
mel = librosa.feature.melspectrogram(sample, sr,hop_length=192,win_length=1024,n_mels=192)

In [94]:
mel.shape

(192, 417)

In [0]:
audio = librosa.feature.inverse.mel_to_audio(mel,sr,hop_length=192,win_length=1152)

In [92]:
ipd.Audio(audio,rate=22050)

### mel -> wave resolution 괜찮은데, mel로 변환할 때, 파라미터(차원)수에 영향을 꽤나 받음